# Markov Chain - Stationary

![Markov Chain](markov_chain.png)

* Based on the notebook:
    * [Markov_Chain_Simulation.ipynb](https://github.com/Suji04/NormalizedNerd/blob/master/Miscellaneous/Markov_Chain_Simulation.ipynb)

In [ ]:
!pip install numpy scipy

In [1]:
import numpy as np

In [2]:
state = {
    0: "Burger",
    1: "Pizza",
    2: "HotDog"
}
state

{0: 'Burger', 1: 'Pizza', 2: 'HotDog'}

## Transition Matrix

<img src="markov_chain_transition_matrix.png" alt="Transition Matrix" style="background-color: black;"/>

$A_{i,j} = P(X_n = j | X_{n-1} = i)$

In [3]:
A = np.array([[0.2, 0.6, 0.2], 
              [0.3, 0, 0.7],
              [0.5, 0, 0.5]])
A

array([[0.2, 0.6, 0.2],
       [0.3, 0. , 0.7],
       [0.5, 0. , 0.5]])

## Random Walk on Markov Chain

In [4]:
n = 15
curr_state = 0
print(state[curr_state], "--->", end=" ")

while n-1:
    curr_state = np.random.choice([0, 1, 2], p=A[curr_state])
    print(state[curr_state], "--->", end=" ")
    n -= 1
print("stop")

Burger ---> Pizza ---> HotDog ---> HotDog ---> HotDog ---> Burger ---> Pizza ---> HotDog ---> HotDog ---> HotDog ---> HotDog ---> HotDog ---> Burger ---> Pizza ---> HotDog ---> stop


## Approach 1: Monte Carlo

In [5]:
steps = 10**6
start_state = 0
curr_state = start_state
pi = np.zeros(3)
pi[start_state] = 1

i = 0
while i < steps:
    curr_state = np.random.choice([0, 1, 2], p=A[curr_state])
    pi[curr_state] += 1
    i += 1

print("π = ", pi/steps)

π =  [0.352167 0.211634 0.4362  ]


## Approach 2: Repeated Matrix Multiplication

In [6]:
steps = 10**3
A_n = A

i = 0
while i < steps:
    A_n = np.matmul(A_n, A)
    i += 1

print("A^n = \n", A_n, "\n")
print("π = ", A_n[0])

A^n = 
 [[0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]] 

π =  [0.35211268 0.21126761 0.43661972]


## Approach 3: Finding Left Eigen Vectors

In [7]:
import scipy.linalg

values, left = scipy.linalg.eig(A, right=False, left=True)

print("left eigen vectors = \n", left, "\n")
print("eigen value = \n", values)

left eigen vectors = 
 [[-0.58746336+0.j         -0.16984156-0.35355339j -0.16984156+0.35355339j]
 [-0.35247801+0.j          0.67936622+0.j          0.67936622-0.j        ]
 [-0.72845456+0.j         -0.50952467+0.35355339j -0.50952467-0.35355339j]] 

eigen value = 
 [ 1.  +0.j        -0.15+0.3122499j -0.15-0.3122499j]


In [10]:
pi = left[:,0]
pi_normalized = [(x/np.sum(pi)).real for x in pi]
print("π =", pi_normalized)

π = [0.3521126760563379, 0.21126760563380298, 0.43661971830985913]


### $P(🍕 \rightarrow 🌭 \rightarrow 🌭 \rightarrow 🍔) = ?$

$P(X_0 = 🍕, X_1 = 🌭, X_2 =🌭, X_3 =🍔)$

$P(X_0 = 🍕) \space P(X_1 = 🌭 | X_0 = 🍕) \space P(X_2 =🌭 | X_1 = 🌭) \space P(X_3 = 🍔 | X_2 =🌭)$

In [33]:
from IPython.display import display, Math

def print_beatiful(seq: list, prob: float):
    if len(seq) > 0 and isinstance(seq[0], str):
        seq = text_to_number(seq)
    
    emojis = []
    latex = "$P("
    for i, element in enumerate(seq):
        if 0 == element:
            latex += f"X_{i} \\rightarrow 🍔"
        elif 1 == element:
            latex += f"X_{i} \\rightarrow 🍕"
        elif 2 == element:
            latex += f"X_{i} \\rightarrow 🌭"
        else:
            raise Exception(f"{element} not supported!")
        if i < len(seq) - 1:
            latex += " \space "
    latex += ") = %g$"
    display(Math(latex % prob))
        
def text_to_number(seq: list):
    state_reverse = {
        "burger": 0,
        "pizza": 1,
        "hotdog": 2
    }
    return [state_reverse[i.lower()] for i in seq]

def find_prob(seq: list, A: np.ndarray, pi: list):
    if len(seq) > 0 and isinstance(seq[0], str):
        seq = text_to_number(seq)
    
    start_state = seq[0]
    prob = pi[start_state]
    prev_state, curr_state = start_state, start_state
    for i in range(1, len(seq)):
        curr_state = seq[i]
        prob *= A[prev_state][curr_state]
        prev_state = curr_state
    return prob

walk_markov = [1, 2, 2, 0]
prob = find_prob(walk_markov, A, pi_normalized)
print_beatiful(walk_markov, prob)

<IPython.core.display.Math object>

In [34]:
walk_markov = ["Burger", "HotDog", "Pizza", "Pizza"]
prob = find_prob(walk_markov, A, pi_normalized)
print_beatiful(walk_markov, prob)

<IPython.core.display.Math object>

In [36]:
walk_markov = ["Burger", "Burger", "Pizza", "HotDog"]
prob = find_prob(walk_markov, A, pi_normalized)
print_beatiful(walk_markov, prob)

<IPython.core.display.Math object>

In [37]:
walk_markov = ["Burger", "Burger"]
prob = find_prob(walk_markov, A, pi_normalized)
print_beatiful(walk_markov, prob)

<IPython.core.display.Math object>